# `pandas` part 1

This week we learn about the following five areas of pandas:

- Data I/O
- Understanding the `DataFrame` and `Series`
- Indexing and Slicing
- First look at the data
- Summary functions

## Importing Libraries

In python, we use the `import` statement to "load" code from other libraries. This is similar to the `library()` function in `R`.

Note we can also use `import library as lib` to rename the imported library. Although it makes for less typing, I recommend sticking to widely accepted abbreviations.

In [1]:
import pandas as pd

pd.__version__ # you can check the version of any library with this command
               # if you get anything older than 0.25.0, try updating pandas with: conda update pandas

'0.25.3'

# Reading in Data

For this lecture we use the BES data. Relevant data files can be downloaded here:
